In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/sMRI_Axial3.zip

Archive:  /content/drive/MyDrive/sMRI_Axial3.zip
   creating: sMRI_Axial3/
   creating: sMRI_Axial3/ASD/
  inflating: sMRI_Axial3/ASD/CALTECH_51456_axial_1.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51456_axial_2.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51456_axial_3.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51457_axial_1.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51457_axial_2.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51457_axial_3.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51458_axial_1.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51458_axial_2.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51458_axial_3.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51459_axial_1.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51459_axial_2.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51459_axial_3.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51460_axial_1.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51460_axial_2.png  
  inflating: sMRI_Axial3/ASD/CALTECH_51460_axial_3.png  
  inflating: sMRI_Axial3/ASD/CALTECH_514

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# ---------------------
# 1. Load & Preprocess Image Data
# ---------------------
image_size = (128, 128)  # Resize all images
batch_size = 32

data_dir = "sMRI_Axial3"  # Adjust if needed

train_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=image_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=image_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=42
)

# Normalize images
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# ---------------------
# 2. Define CNN Model
# ---------------------
def build_cnn():
    inputs = keras.Input(shape=(128, 128, 3))  # RGB images

    x = layers.Conv2D(32, kernel_size=3, activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(64, kernel_size=3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)  # Binary classification

    return keras.Model(inputs, outputs)

cnn_model = build_cnn()

# ---------------------
# 3. Define Physics-Informed Loss (PINN)
# ---------------------
def physics_loss(y_true, y_pred):
    """
    Custom physics-informed loss function.
    Constraints:
    1. Brain Symmetry Regularization
    2. Morphometric Constraints
    """
    classification_loss = keras.losses.BinaryCrossentropy()(y_true, y_pred)

    # Constraint 1: Simulated Brain Symmetry
    symmetry_loss = tf.reduce_mean(tf.abs(y_pred[:, 0] - y_pred[:, 0][::-1]))

    # Constraint 2: Regularization for Stability
    regularization = tf.reduce_mean(tf.abs(y_pred - 0.5))

    total_loss = classification_loss + 0.1 * symmetry_loss + 0.05 * regularization
    return total_loss

# ---------------------
# 4. Train Model with PINN Loss
# ---------------------
cnn_model.compile(optimizer="adam", loss=physics_loss, metrics=["accuracy"])

cnn_model.fit(train_ds, epochs=10, validation_data=val_ds)

# ---------------------
# 5. Evaluate Model
# ---------------------
test_loss, test_acc = cnn_model.evaluate(val_ds)
print(f"Test Accuracy: {test_acc:.4f}")


Found 3324 files belonging to 2 classes.
Using 2660 files for training.
Found 3324 files belonging to 2 classes.
Using 664 files for validation.
Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.5227 - loss: 0.8424 - val_accuracy: 0.5813 - val_loss: 0.6889
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.5995 - loss: 0.6785 - val_accuracy: 0.6416 - val_loss: 0.6350
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.7157 - loss: 0.5964 - val_accuracy: 0.6039 - val_loss: 0.7561
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.8315 - loss: 0.4251 - val_accuracy: 0.8238 - val_loss: 0.4374
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.9321 - loss: 0.2405 - val_accuracy: 0.9021 - val_loss: 0.3169
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.9734 - loss: 0.1632 - val_accuracy: 0.9232 - val_loss: 0.2706
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.9815 - loss: 0.1345 - val_accur